In [ ]:
import sys
import os
from pathlib import Path

# importディレクトリの追加
# sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
print(sys.path)

# プロキシの設定
# os.environ['HTTP_PROXY'] = ''
# os.environ['HTTPS_PROXY'] = ''

%matplotlib inline

In [ ]:
!nvidia-smi

# Automatic Differentiation with `torch.autograd`

- NNを訓練する際, 基本的には誤差逆伝播(back propagation)が使用される
    - backpropでは, モデルのパラメータは **損失関数に対するその変数の微分値(勾配)に応じて調整される**
    
- PyTorchには勾配の値を計算するために, `torch.autograd`という微分エンジンが組み込まれており, 計算グラフの勾配を自動計算できる

- 以下, シンプルな1層のネットワークを想定した例で説明

In [1]:
import torch

x = torch.ones(5)
y = torch.zeros(3)

# 最適化するパラメータ
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)

# 計算の定義
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Back Propagation

- 計算グラフを構築する際に, テンソルに適用する関数は[Functionクラスのオブジェクト](https://pytorch.org/docs/stable/autograd.html#function)が適用される
- Functionクラスオブジェクトには, **順伝播と逆伝播の計算処理の定義が含まれている**
- 勾配計算の定義はテンソルの`grad_fn`プロパティに格納される
- 偏微分値を求めるためには, `backward()`を実行し, `w.grad`と`b.grad`の値を導出する

- **gradは計算グラフのleaf nodeかつ, reuires_grad=Trueのテンソルのみで求めることができる**
- すべての変数で勾配計算が行えるわけではない
- **勾配計算は各計算グラフに対して, backwardを1回のみ実行できる**
    - **同じ計算グラフに複数回のbackwardを実行する場合, retain_graph=Trueをbackwardの引数に指定する必要がある**

In [3]:
print(f'Gradient function for z = {z.grad_fn}')
print(f'Gradient function for loss = {loss.grad_fn}')

loss.backward()
print(w.grad)
print(b.grad)

# 2回目の誤差逆伝播を実行
# loss.backward() => "Runtime Error: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time."

Gradient function for z = <AddBackward0 object at 0x7fadac31d970>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7fadac31da60>


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

## 勾配計算をしない方法

- 勾配計算が不要な場合は以下の方法を用いて無効化できる
  - `torch.no_grad()`ブロックを作成し, それらのコードを記載する
  - `detach()`を使用する  

 
- 勾配計算や追跡を不能にしたいケースは以下のようなもの

  - ネットワークパラメータの一部を固定したい(**ファインチューニング等**)
  - 順伝播の計算スピードを高速化したい

In [4]:
z1 = torch.matmul(x, w) + b
print(z1.requires_grad)

with torch.no_grad():
    z2 = torch.matmul(x, w) + b
    print(z2.requires_grad)

z3 = torch.matmul(x, w) + b
z_det = z3.detach()
print(z_det.requires_grad)

True
False
False


# 計算グラフについて補足

- autogradはテンソルとそれらに対する演算処理を[`Function`](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)を構成単位として、DAG（a directed acyclic graph）の形で保存したグラフ

- DAGにおいて, leafは入力テンソル, rootは出力テンソルとなる
- このグラフをrootから各leafまで**微分の連鎖率で追跡する**ことで, 各変数に対する偏微分の値を求めることができる

- `.backward()`がDAGのrootテンソルに対して実行されると, autogradは以下を行う
  - 各変数の`.grad_fn()`を計算する
  - 各変数の`.grad`プロパティに微分値を代入する
  - 微分の連鎖率を使用して, 各leafの微分値を求める

## テンソルに対する勾配とヤコビ行列

- 損失関数の出力はスカラ値の場合が多いが, テンソルが返されるケースもある
- このような場合, PyTorchでは実際の勾配ではなく, **ヤコビ行列(Jacobian matrix)** を計算することができる

- ヤコビ行列そのものを計算する代わりに, PytorchではJacobian Productを入力ベクトル $ v $ に対して計算する
  - `backward()`を引数なしに実行するのは, **`backward(torch.tensor(1.0))`実行しているのと同じ**  


- PyTorchでは`backward`を実行すると,**勾配を蓄積する**
  - 計算グラフの全leafのgradには, **勾配が足し算される**

In [7]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)

# 1回目のbackprop
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)

# 2回目のbackprop
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)  # => 勾配が足し算される

# 勾配を0リセット
inp.grad.zero_()

out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
